In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

import re
import pandas as pd
import time
import requests

## 멜론Top100크롤링

In [ ]:
top100_number=[]

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.melon.com/chart/index.htm")
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')

songs=soup.select('div.wrap > a.btn.button_icons.type03.song_info')
for i in range(len(songs)):
        href=songs[i].attrs['href']
        top100_number.append(re.sub('[^0-9]','',href))

In [ ]:
len(top100_number)

100

In [ ]:
driver=webdriver.Chrome()
lyric_lst=[]
cnt=0
for i in top100_number:
    url="https://www.melon.com/song/detail.htm?songId="+i
    headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,'html.parser')
    try:
        lyric_lst.append(soup.select('div.lyric')[0].text)
    except:
        cnt+=1
    time.sleep(2)

In [ ]:
len(lyric_lst)

100

In [ ]:
top100_title=[]
top100_singer=[]
driver = webdriver.Chrome()
url='https://www.melon.com/chart/index.htm'
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
r=requests.get(url,headers=headers)
soup=BeautifulSoup(r.text,'html.parser')

for i in range(0,100):
    title=soup.select('div.ellipsis.rank01')[i].text
    top100_title.append(re.sub('\n','',title))
    singer=soup.select('div.ellipsis.rank02')[i].text
    singer=re.sub('\n','',singer)
    top100_singer.append(singer[:int(len(singer)/2)])

In [ ]:
print(len(top100_singer),len(top100_title))

100 100


In [ ]:
top100=pd.DataFrame({'singer':top100_singer,
                    'title':top100_title,
                    'lyrics':lyric_lst})
top100.to_csv('./data/top100.csv',encoding='utf-8-sig',index=False)

In [ ]:
top30=pd.DataFrame({'singer':top100_singer[:30],
                    'title':top100_title[:30],
                    'lyrics':lyric_lst[:30]})

In [ ]:
top30.to_csv('./data/top30.csv',encoding='utf-8-sig',index=False)

## 전처리과정
번역

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import os
import sys
import urllib.request
import requests
import time
import nltk

import googletrans
from sklearn.model_selection import train_test_split

In [ ]:
def checkKorean(str):
    pattern = '[ㄱ-ㅎ가-힣\s]+'
    if re.match(pattern, str):
        return True
    else:
        return False

In [ ]:
translator=googletrans.Translator()
trans=[]
for i,lyric in enumerate(top30['lyrics']):
    split_sentences = re.compile("[ㄱ-ㅎ가-힣\s]+|[a-zA-Z\s']+").findall(lyric)
    sen=''
    for sentence in split_sentences:
        check=checkKorean(sentence)
        if not check:
            try:
                en2ko=translator.translate(sentence,dest='ko',src='en')
                sen+=en2ko.text
            except:
                sen+=sentence
        else:
            sen+=sentence
    trans.append(sen)
len(trans)

30

In [ ]:
lyrics=[]
for lyric in trans:
    sub=re.sub("[0-9a-zA-Z'.,?!()&^%$#\t\r\n\\s]+",' ',lyric)
    sub=sub.strip()
    lyrics.append(re.sub(' +', ' ', sub))
lyrics[0]

'우우 우 우우 우 우우 우 우 우스 테이에서 미들 같은 넌 수수께끼를 원하지 않습니다말해줘 백로라고 말하면 라고 말합니다아침은 너무 멀어그러니 라고 말합니다훌쩍 커버렸어함께한 기억처럼널 보는 내 마음은어느새 여름 지나 가을기다렸지 이 모든 시간에 내가 원하는 사람처럼 누군가를 원해 날 보고 웃었지만당신은 지금 나에 대해 생각하십니까 그래 항상 그래야 할 시간이 없어서 잃을 시간이 없었어내 길었던 하루난 보고 싶어 고마워고마워고마워울린 심장 고마워고마워고마워나는 잃을 것이 없다널 좋아한다고우 우아 우아라고마워고마워고마워울린 심장 고마워고마워고마워하지만 난 너처럼 가운데에 머물고 싶지 않아수수께끼를 원하지 않습니다말해줘 백로라고 말하면 라고 말합니다아침은 너무 멀어그러니 내가 이것을 걷고 싶지 않아미로다 아는 건 아니어도바라던 대로말해줘 백로가 가 당신을 원한다고 말합니다 당신이 누구에게나 이라고 말하기를 원합니다너를 상상했지항상 닿아있던처음 느낌 그대로 난기다렸지 이 모든 시간에 나는 잃을 것이 없었다널 좋아한다고우 우아 우아라고마워고마워고마워울린 심장 고마워고마워고마워하지만 난 너처럼 가운데에 머물고 싶지 않아수수께끼를 원하지 않습니다말해줘 백로라고 말하면 라고 말합니다아침은 너무 멀어그러니 내가 이것을 걷고 싶지 않아미로다 아는 건 아니어도바라던 대로말해줘 백로가 가 당신을 원한다고 말합니다'

In [ ]:
top30['lyrics']=lyrics

In [ ]:
top30.head()

,singer,title,lyrics
0,NewJeans,Ditto,우우 우 우우 우 우우 우 우 우스 테이에서 미들 같은 넌 수수께끼를 원하지 않습니...
1,NewJeans,OMG,이 노래는 그것당신에 대해 아가비 만만 너희 너 내가 힘들 때울 것 같을 때기운도 ...
2,NewJeans,Hype boy,아기 내가 너무 미쳤다빠져버리는 은 내가 당신을 느끼게했습니다너도 말해줄래누가 내게...
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만소녀 당신은 당신이 그 분위기를 가지고 있다는 것을 알아야합니...
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야당연히 해야 할 일이야나에겐 우리가 지금 순위야안전한 유리병을...


맞춤법 교정

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

from wordcloud import WordCloud
from konlpy.tag import Okt
from hanspell import spell_checker

from soynlp.normalizer import *
import urllib.request
from soyspacing.countbase import CountSpace,RuleDict
import soyspacing

In [ ]:
lyrics=[]
for lyric in top30['lyrics']:
    lyrics.append(repeat_normalize(lyric,num_repeats=1))
len(lyrics)

30

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")
model=CountSpace()
model.train('./2016-10-20.txt')

all tags length = 1909391 --> 1671744, (num_doc = 30090))

In [ ]:
result=[]
for lyric in lyrics:
    sub,tags=model.correct(lyric,verbose=False,force_abs_threshold=0.3,nonspace_threshold=-0.3,space_threshold=0.3,min_count=10)
    result.append(sub)
len(result)

30

In [ ]:
lyrics=[]
cnt=0
error=0
for i,lyric in enumerate(result):
    try:
        if len(lyric)>500:
            sub=''
            spelled_sent = spell_checker.check(lyric[:500])
            spelled_sent=spelled_sent.as_dict()
            sub+=spelled_sent['checked']
            
            spelled_sent = spell_checker.check(lyric[500:])
            spelled_sent=spelled_sent.as_dict()
            sub+=spelled_sent['checked']
            
            lyrics.append(sub)

        else:
            spelled_sent=spell_checker.check(lyric)
            spelled_sent=spelled_sent.as_dict()
            lyrics.append(spelled_sent['checked'])
    except:
        lyrics.append(lyric)
        error+=1
    cnt+=1
    if cnt%100==0:
        print(cnt)

In [ ]:
result=[]
for lyric in lyrics:
    result.append(repeat_normalize(lyric,num_repeats=1))
len(result)

30

In [ ]:
top30['lyrics']=result
top30.head()

,singer,title,lyrics
0,NewJeans,Ditto,우우 우 우우 우 우우 우 우 우스 테이에서 미들 같은 넌 수수께끼를 원하지 않습니...
1,NewJeans,OMG,이 노래는 그것 당신에 대해 아이비 만만 너희 너 내가 힘들 때울 것 같을 때 기운...
2,NewJeans,Hype boy,아기 내가 너무 미쳤다 빠져버리는 은 내가 당신을 느끼게 했습니다 너도 말해줄래 누...
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 당신은 당신이 그 분위기를 가지고 있다는 것을 알아야 ...
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 지금 순위야 안전한 유...


중복단어제거

In [ ]:
lst=[]
for lyrics in top30['lyrics']:
    lyric=lyrics.split(' ')
    sub=[]
    for word in lyric:
        if word not in sub or sub.count(word)<3:
            sub.append(word)
    lst.append(sub)
print(len(lst))

30


In [ ]:
lyrics=[]
for word_lst in lst:
    sen=''
    sen=' '.join(word_lst)
    lyrics.append(sen)
print(len(lyrics))

30


In [ ]:
top30['lyrics']=lyrics
top30.head()

,singer,title,lyrics
0,NewJeans,Ditto,우우 우 우우 우 우우 우 우스 테이에서 미들 같은 넌 수수께끼를 원하지 않습니다 ...
1,NewJeans,OMG,이 노래는 그것 당신에 대해 아이비 만만 너희 너 내가 힘들 때울 것 같을 때 기운...
2,NewJeans,Hype boy,아기 내가 너무 미쳤다 빠져버리는 은 내가 당신을 느끼게 했습니다 너도 말해줄래 누...
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 당신은 당신이 그 분위기를 가지고 있다는 것을 알아야 ...
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 지금 순위야 안전한 유...


불용어제거

In [ ]:
f = open("stopword.txt", "rt",encoding='utf-8')
stopword = []
while True:
    line = f.readline().strip()
    if not line: break
    stopword.append(line)

In [ ]:
lst=[]
for lyric in top30['lyrics']:
    lyric=lyric.split(' ')
    sub=[]

    sub=[word for word in lyric if not word in stopword]
    lst.append(sub)
print(len(lst))

30


In [ ]:
lyrics=[]
for word_lst in lst:
    sen=''
    sen=' '.join(word_lst)
    lyrics.append(sen)
print(len(lyrics))

30


In [ ]:
top30['lyrics']=lyrics
top30.head()

,singer,title,lyrics
0,NewJeans,Ditto,우스 테이에서 미들 수수께끼를 원하지 않습니다 말해줘 백로라고 말하면 하고 말합니다...
1,NewJeans,OMG,노래는 그것 당신에 대해 아이비 만만 힘들 때울 같을 기운도 나지 않을 그것 걱정하...
2,NewJeans,Hype boy,아기 미쳤다 빠져버리는 은 당신을 느끼게 했습니다 너도 말해줄래 누가 뭐라 든 남들...
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 분위기를 가지고 있다는 것을 알아야 합니다에 미소는 미...
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 순위야 안전한 유리병을...


In [ ]:
top30.to_csv('./data/top30_pre.csv',encoding='utf-8-sig',index=False)

## 사전기반
knu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from wordcloud import WordCloud

from konlpy import tag
from konlpy.corpus import kolaw
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능 #형태소 분석기 호출

from nltk.tokenize import sent_tokenize
from collections import Counter #데이터의 개수를 정리할 수 있는 모듈
from konlpy.utils import pprint #유니코드 문자 출력

import warnings
warnings.filterwarnings('ignore')

In [ ]:
top30=pd.read_csv('./data/top30_pre.csv',encoding='utf-8')
top30.head(2)

,singer,title,lyrics
0,NewJeans,Ditto,우스 테이에서 미들 수수께끼를 원하지 않습니다 말해줘 백로라고 말하면 하고 말합니다...
1,NewJeans,OMG,노래는 그것 당신에 대해 아이비 만만 힘들 때울 같을 기운도 나지 않을 그것 걱정하...


In [ ]:
def LyricsWordCount(arrList):

    pos_word = pd.read_csv('./pos_pol_word.txt', index_col = 0)
    pos_word = pd.DataFrame(pos_word).reset_index()
    pos_word.columns = ['pos_word']

    neg_word = pd.read_csv('./neg_pol_word.txt', index_col = 0)
    neg_word = pd.DataFrame(neg_word).reset_index()
    neg_word.columns = ['neg_word']

    list = []
    result = []
    knu_num=[]
    i=1

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수 
        
        # 긍정 점수 구하기
        for pos_text in pos_word['pos_word']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in neg_word['neg_word']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: neg_count += 0

        # 감성 점수 공식
        if pos_count==0 or neg_count==0:
          num=0
        else:
          num=pos_count/(pos_count+neg_count)
        knu_num.append(num)

        # 긍, 부정 결과
        if num>=0.5:
          result='긍정'
        else:
          result='부정'

        srt = f"{i}번 노래 결과 -> 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {round(num,3)}  {result}"
        i = i + 1
        list.append(srt)
    return list,knu_num

In [ ]:
pos_neg,knu_num=LyricsWordCount(top30['lyrics'])
pos_neg

['1번 노래 결과 -> 긍정 : 5, 부정 : 5 , 감성 점수 : 0.5  긍정',
 '2번 노래 결과 -> 긍정 : 2, 부정 : 4 , 감성 점수 : 0.333  부정',
 '3번 노래 결과 -> 긍정 : 4, 부정 : 2 , 감성 점수 : 0.667  긍정',
 '4번 노래 결과 -> 긍정 : 9, 부정 : 5 , 감성 점수 : 0.643  긍정',
 '5번 노래 결과 -> 긍정 : 12, 부정 : 6 , 감성 점수 : 0.667  긍정',
 '6번 노래 결과 -> 긍정 : 2, 부정 : 14 , 감성 점수 : 0.125  부정',
 '7번 노래 결과 -> 긍정 : 3, 부정 : 10 , 감성 점수 : 0.231  부정',
 '8번 노래 결과 -> 긍정 : 6, 부정 : 7 , 감성 점수 : 0.462  부정',
 '9번 노래 결과 -> 긍정 : 5, 부정 : 1 , 감성 점수 : 0.833  긍정',
 '10번 노래 결과 -> 긍정 : 4, 부정 : 5 , 감성 점수 : 0.444  부정',
 '11번 노래 결과 -> 긍정 : 7, 부정 : 2 , 감성 점수 : 0.778  긍정',
 '12번 노래 결과 -> 긍정 : 3, 부정 : 5 , 감성 점수 : 0.375  부정',
 '13번 노래 결과 -> 긍정 : 4, 부정 : 8 , 감성 점수 : 0.333  부정',
 '14번 노래 결과 -> 긍정 : 4, 부정 : 2 , 감성 점수 : 0.667  긍정',
 '15번 노래 결과 -> 긍정 : 6, 부정 : 2 , 감성 점수 : 0.75  긍정',
 '16번 노래 결과 -> 긍정 : 6, 부정 : 2 , 감성 점수 : 0.75  긍정',
 '17번 노래 결과 -> 긍정 : 6, 부정 : 8 , 감성 점수 : 0.429  부정',
 '18번 노래 결과 -> 긍정 : 5, 부정 : 9 , 감성 점수 : 0.357  부정',
 '19번 노래 결과 -> 긍정 : 7, 부정 : 10 , 감성 점수 : 0.412  부정',
 '20번 노래 결과 -> 긍정 : 4

In [ ]:
top30['knu']=knu_num
top30.head()

,singer,title,lyrics,knu
0,NewJeans,Ditto,우스 테이에서 미들 수수께끼를 원하지 않습니다 말해줘 백로라고 말하면 하고 말합니다...,0.500000
1,NewJeans,OMG,노래는 그것 당신에 대해 아이비 만만 힘들 때울 같을 기운도 나지 않을 그것 걱정하...,0.333333
2,NewJeans,Hype boy,아기 미쳤다 빠져버리는 은 당신을 느끼게 했습니다 너도 말해줄래 누가 뭐라 든 남들...,0.666667
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 분위기를 가지고 있다는 것을 알아야 합니다에 미소는 미...,0.642857
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 순위야 안전한 유리병을...,0.666667


kosac

In [ ]:
import pandas as pd
import numpy as np

import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Kkma, Komoran, Okt, Hannanum  #Mecab은 윈도우에서 작동 불가능

from konlpy import tag
from konlpy.corpus import kolaw
# from konlpy.utils import csvwrite, pprint

from nltk.tokenize import sent_tokenize

In [ ]:
kosac=pd.read_csv('./polarity.csv',encoding='utf-8')

In [ ]:
kkma=Kkma()

data=top30['lyrics'].to_string()
kkma_data=kkma.pos(data)

In [ ]:
kkma_list=[]

for i in range(len(kkma_data)):
    word=kkma_data[i][0]+'/'+kkma_data[i][1]
    kkma_list.append(word)
    i+=1

In [ ]:
text = ''.join(str(i) for i in kkma_list)

text_list = []

numList = re.findall(r'\d+',text)

In [ ]:
numList_pos=[]
for i in numList:
  if(text.find(f'{i}') > -1):
    num = text.find(f'{i}')
    numList_pos.append(num)

In [ ]:
for i in range(len(numList_pos)):
  if(i == len(numList_pos)-1):
    text_2 = text[numList_pos[i]:len(text)]
  else:
    text_2 = text[numList_pos[i]:numList_pos[i+1]]
  text_list.append(text_2)

In [ ]:
df=pd.DataFrame(text_list)
df.columns=['word']

In [ ]:
kosac_pos=kosac[kosac['max.value']=='POS']
kosac_neg=kosac[kosac['max.value']=='NEG']

In [ ]:
def LyricsWordCount(arrList):
    kosac_num=[]
    list = []
    i=1

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수
        
        # 긍정 점수 구하기
        for pos_text in kosac_pos['ngram']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in kosac_neg['ngram']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: neg_count += 0

        # 감성 점수 공식
        num = pos_count - neg_count

        # 감성 점수 공식
        if pos_count == 0 or neg_count == 0:
            num = 0
        else:
            num = pos_count/(pos_count + neg_count)
        
        kosac_num.append(num)

        # 긍, 부정 결과
        if num >= 0.5:
            result = '긍정'
        elif num < 0.5:
            result = '부정'


        srt = f"{i}번 노래 결과 : 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {round(num,3)}  {result}"
        i = i + 1

        list.append(srt)
    return list,kosac_num


In [ ]:
pos_result,kosac_num = LyricsWordCount(df['word'])

for pos in pos_result:
    print(pos)

1번 노래 결과 : 긍정 : 9, 부정 : 4 , 감성 점수 : 0.692  긍정
2번 노래 결과 : 긍정 : 7, 부정 : 9 , 감성 점수 : 0.438  부정
3번 노래 결과 : 긍정 : 11, 부정 : 11 , 감성 점수 : 0.5  긍정
4번 노래 결과 : 긍정 : 9, 부정 : 7 , 감성 점수 : 0.562  긍정
5번 노래 결과 : 긍정 : 5, 부정 : 9 , 감성 점수 : 0.357  부정
6번 노래 결과 : 긍정 : 8, 부정 : 5 , 감성 점수 : 0.615  긍정
7번 노래 결과 : 긍정 : 5, 부정 : 6 , 감성 점수 : 0.455  부정
8번 노래 결과 : 긍정 : 6, 부정 : 13 , 감성 점수 : 0.316  부정
9번 노래 결과 : 긍정 : 7, 부정 : 9 , 감성 점수 : 0.438  부정
10번 노래 결과 : 긍정 : 9, 부정 : 10 , 감성 점수 : 0.474  부정
11번 노래 결과 : 긍정 : 6, 부정 : 9 , 감성 점수 : 0.4  부정
12번 노래 결과 : 긍정 : 8, 부정 : 4 , 감성 점수 : 0.667  긍정
13번 노래 결과 : 긍정 : 12, 부정 : 7 , 감성 점수 : 0.632  긍정
14번 노래 결과 : 긍정 : 3, 부정 : 4 , 감성 점수 : 0.429  부정
15번 노래 결과 : 긍정 : 7, 부정 : 7 , 감성 점수 : 0.5  긍정
16번 노래 결과 : 긍정 : 8, 부정 : 7 , 감성 점수 : 0.533  긍정
17번 노래 결과 : 긍정 : 7, 부정 : 13 , 감성 점수 : 0.35  부정
18번 노래 결과 : 긍정 : 6, 부정 : 9 , 감성 점수 : 0.4  부정
19번 노래 결과 : 긍정 : 10, 부정 : 8 , 감성 점수 : 0.556  긍정
20번 노래 결과 : 긍정 : 7, 부정 : 13 , 감성 점수 : 0.35  부정
21번 노래 결과 : 긍정 : 8, 부정 : 2 , 감성 점수 : 0.8  긍정
22번 노래 결과 : 긍정 : 5, 부정 : 5

In [ ]:
top30['kosac']=kosac_num
top30.head()

,singer,title,lyrics,knu,kosac
0,NewJeans,Ditto,우스 테이에서 미들 수수께끼를 원하지 않습니다 말해줘 백로라고 말하면 하고 말합니다...,0.500000,0.692308
1,NewJeans,OMG,노래는 그것 당신에 대해 아이비 만만 힘들 때울 같을 기운도 나지 않을 그것 걱정하...,0.333333,0.437500
2,NewJeans,Hype boy,아기 미쳤다 빠져버리는 은 당신을 느끼게 했습니다 너도 말해줄래 누가 뭐라 든 남들...,0.666667,0.500000
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 분위기를 가지고 있다는 것을 알아야 합니다에 미소는 미...,0.642857,0.562500
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 순위야 안전한 유리병을...,0.666667,0.357143


knu, kosac 평균

In [ ]:
top30['result']=(top30['knu']+top30['kosac'])/2
top30.head()

,singer,title,lyrics,knu,kosac,result
0,NewJeans,Ditto,우스 테이에서 미들 수수께끼를 원하지 않습니다 말해줘 백로라고 말하면 하고 말합니다...,0.500000,0.692308,0.596154
1,NewJeans,OMG,노래는 그것 당신에 대해 아이비 만만 힘들 때울 같을 기운도 나지 않을 그것 걱정하...,0.333333,0.437500,0.385417
2,NewJeans,Hype boy,아기 미쳤다 빠져버리는 은 당신을 느끼게 했습니다 너도 말해줄래 누가 뭐라 든 남들...,0.666667,0.500000,0.583333
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 분위기를 가지고 있다는 것을 알아야 합니다에 미소는 미...,0.642857,0.562500,0.602679
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 순위야 안전한 유리병을...,0.666667,0.357143,0.511905


In [ ]:
def pos_neg(x):
    if x['result']>=0.5:
        return '1'
    else:
        return '0'

top30['pos']=top30.apply(pos_neg,axis=1)
top30.head()

,singer,title,lyrics,knu,kosac,result,pos
0,NewJeans,Ditto,우스 테이에서 미들 수수께끼를 원하지 않습니다 말해줘 백로라고 말하면 하고 말합니다...,0.500000,0.692308,0.596154,1
1,NewJeans,OMG,노래는 그것 당신에 대해 아이비 만만 힘들 때울 같을 기운도 나지 않을 그것 걱정하...,0.333333,0.437500,0.385417,0
2,NewJeans,Hype boy,아기 미쳤다 빠져버리는 은 당신을 느끼게 했습니다 너도 말해줄래 누가 뭐라 든 남들...,0.666667,0.500000,0.583333,1
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 분위기를 가지고 있다는 것을 알아야 합니다에 미소는 미...,0.642857,0.562500,0.602679,1
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 순위야 안전한 유리병을...,0.666667,0.357143,0.511905,1


In [ ]:
top30

,singer,title,lyrics,knu,kosac,result,pos
0,NewJeans,Ditto,우스 테이에서 미들 수수께끼를 원하지 않습니다 말해줘 백로라고 말하면 하고 말합니다...,0.500000,0.692308,0.596154,1
1,NewJeans,OMG,노래는 그것 당신에 대해 아이비 만만 힘들 때울 같을 기운도 나지 않을 그것 걱정하...,0.333333,0.437500,0.385417,0
2,NewJeans,Hype boy,아기 미쳤다 빠져버리는 은 당신을 느끼게 했습니다 너도 말해줄래 누가 뭐라 든 남들...,0.666667,0.500000,0.583333,1
3,태양,VIBE (feat. Jimin of BTS),말론 표현할 수 없지만 소녀 분위기를 가지고 있다는 것을 알아야 합니다에 미소는 미...,0.642857,0.562500,0.602679,1
4,윤하 (YOUNHA),사건의 지평선,생각이 많은 건 말이야 당연히 해야 할 일이야 나에겐 우리가 순위야 안전한 유리병을...,0.666667,0.357143,0.511905,1
5,투모로우바이투게더,Sugar Rush Ride,생각은 숨을 멈췄어 돌아올 다짐 따윈 한순간 풀어 헤쳐져 네겐 가까이 오라 속삭여 ...,0.125000,0.615385,0.370192,0
6,부석순 (SEVENTEEN),파이팅 해야지 (Feat. 이영지),파이팅 해야지 파이팅 아뿔싸 일어나야지 아침인데 눈 감았다 뜨니 해가 중천인데 아침...,0.230769,0.454545,0.342657,0
7,LE SSERAFIM (르세라핌),ANTIFRAGILE,가시밭길 위로 라이딩은 향상시켰다 거짓으로 가득 찬 파티 가렵지도 않아내 뒤에 말들...,0.461538,0.315789,0.388664,0
8,NewJeans,Attention,너와 맘이 보이지 한참을 쳐다봐 가까이 다가가 볼 수 있습니다 상용기가 생겼지 이미...,0.833333,0.437500,0.635417,1
9,임영웅,사랑은 늘 도망가,눈물이 난다 길을 걸으면 사람 손길이 생각이 난다 붙잡지 못하고 가슴만 떨었지 아름...,0.444444,0.473684,0.459064,0


In [ ]:
top30.to_csv('./data/top30_dict.csv',encoding='utf-8-sig',index=False)